In [1]:
## GeNNus Simone & Paolo

In [16]:
import torch
import tqdm
import os
import numpy as np
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
DATASET_SIZE = "xs"
DATASET_TYPE = "waveform"

DATASET_FOLDER = f"./data/{DATASET_TYPE}"

DATASET_NUM_SAMPLES_PER_SECOND = 8000
DATASET_NUM_CHANNELS = 1

DATASET_NAME = f"fma_{DATASET_SIZE}_resampled_{DATASET_NUM_SAMPLES_PER_SECOND}_rechanneled_{DATASET_NUM_CHANNELS}"

dataset_path = f"{DATASET_FOLDER}/{DATASET_NAME}"

In [3]:
audio_path_list = []

for path, subdirs, files in os.walk(dataset_path):
    for name in files:
        file_audio_path = os.path.join(path, name)
        #print(file_audio_path)

        if name != '.DS_Store':
            audio_path_list.append(file_audio_path)

audio_path_list = sorted(audio_path_list , reverse= True)
# lista di tutti i path     

In [ ]:
single_tensors = []
labels = []

for p in audio_path_list:
    single_tensors.append(torch.load(p))
    #print(p)
    labels.append(p.split("/")[-2])

single_tensors # --> waveforms
labels # --> etichette

In [6]:
stacked_single_tensors = torch.cat(single_tensors).numpy()

(238000,)

In [9]:
data = pd.DataFrame(stacked_single_tensors)

In [10]:
data.insert(0, "Label", labels, True)

In [ ]:
data

In [13]:
import random 
def train_test_split( dataset , perc_train , set_seed = 69):

    train = dataset.sample(frac= perc_train,random_state = set_seed )
    test  = dataset.drop(train.index)

    return (train , test )


In [14]:
train_dataset , test_dataset = train_test_split( data , perc_train = .9)

In [ ]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import utils
from tqdm import tqdm


X = train_dataset.iloc[: , 1:]
y = train_dataset.iloc[: , :1]

#Implementing cross validation

k = 3
kf = KFold( n_splits=k, random_state= None , shuffle= True )
model = LogisticRegression()

acc_score = []
for j in range(k):
    x_train , x_validation = train_valid_test_split( X , perc_train = .78 , set_seed= j)
    y_train , y_validation = train_valid_test_split( y , perc_train = .78 , set_seed= j)
    
    model.fit(x_train , y_train)
    pred_values = model.predict(x_validation)
    
    acc = accuracy_score( pred_values , y_validation)
    acc_score.append(acc)

avg_acc_score = sum(acc_score)/k

In [72]:
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.2222222222222222, 0.3333333333333333, 0.16666666666666666]
Avg accuracy : 0.24074074074074073


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split( stacked_single_tensors , labels, test_size=0.2, random_state=42 )
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(X_train, Y_train)


In [115]:
LogisticPredictions = logistic.predict( X_test )
Logistic_Accuracy = logistic.score( X_test , Y_test) 

In [ ]:
LogisticPredictions

In [118]:
Logistic_Accuracy

0.1111111111111111

In [119]:
from sklearn.metrics import confusion_matrix
confusion_matrix( Y_test , LogisticPredictions , labels = None, sample_weight=None, normalize=None)

array([[0, 4, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0],
       [0, 4, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0],
       [0, 4, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0]])

In [ ]:
from sklearn.model_selection import KFold
key_fold = KFold( n_splits=4 ) 
key_fold.get_n_splits(X_train)

In [79]:
import pandas as pd
df = pd.DataFrame(stacked_single_tensors)
X = df.iloc[:,:-1]
y = labels


In [ ]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import utils
from tqdm import tqdm
set.seed(69)


X = df.iloc[:,:-1]

#convert y values to categorical values

y = pd.DataFrame(labels)

#Implementing cross validation

k = 3
kf = KFold(n_splits=k, random_state= None , shuffle= True )
model = LogisticRegression()

acc_score = []

for train_index , test_index in tqdm( kf.split(X) ):

    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y.iloc[train_index] , y.iloc[test_index]


    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score( pred_values , y_test)
    acc_score.append(acc)

avg_acc_score = sum(acc_score)/k


accuracy of each fold - [0.14814814814814814, 0.1111111111111111, 0.25925925925925924]
Avg accuracy : 0.1728395061728395
